# ALF Tutorial for $\lambda$ Dynamics Workshop

In [ ]:
import alf
import os, subprocess, shutil
import numpy as np
alf_tutorial_dir=os.getcwd()

## I. Statement of the Problem

ALF estimates free energy differences between physical states drawn from an equilibrium distribution using a histogram based estimator. States with $\lambda>0.99$ are counted as $\lambda=1$. The number of times a state is observed gives its probability $P_i$, which can be converted to a free energy:
$$ G=-kT\ln(P_i)$$

The assumption of equilibrium gives rise to two problems
* If states differ in free energy by more than 2-3 kcal/mol, higher free energy states will not be sampled enough to accurately estimate their probability
* If there are barriers or traps between physical states of height $\Delta G^{\ddagger}$, the rate ($k$) of coming to equilibrium slows down exponentially
$$ k \propto \exp(-\Delta G^{\ddagger}/k_BT)$$

The solution to these problems is to flatten the alchemical space as a function of the alchemical coordinates $\lambda$ to maximize the rate of convergence of the simulations

## II. Test System: 1,4-substituted-benzene Solvation Free Energy

Solvation free energy of 1,4-substituted-benzene. Focus on vacuum side of solvation free energy to obtain quick results.
* 5 substituents at first site
* 6 substituents at second site (omit carboxylic acid due to protonation changes)

Several experimental results are available from early SAMPL challenges

<img src="Figures/14benz/14benz.jpeg" alf="1,4-benzene" width=250/>

Note: site 2 is mislabeled in this diagram. $\lambda_{21}$ is correct, switch $\lambda_{22}$ and $\lambda_{27}$, switch $\lambda_{23}$ and $\lambda_{26}$, and switch $\lambda_{24}$ and $\lambda_{25}$.

JZ Vilseck, X Ding, RL Hayes, & CL Brooks III. Generalizing the Discrete Gibbs Sampler-Based $\lambda$-Dynamics Approach for Multisite Sampling of Many Ligands. JCTC 17:3895–3907 (2021)

DOI: <a href=https://doi.org/10.1021/acs.jctc.1c00176>10.1021/acs.jctc.1c00176</a>

## III. Profiles

Projections of the alchemical free energy landscape along 1-D and 2-D alchemical reaction coordinates are taken to assess whether the free energy landscape is flat
* 1-D profiles bin as a function of a single variable (e.g. $\lambda_{1}$)
* 2-D profiles bin as a function of two variables (e.g. $\lambda_{1}$ and $\lambda_{2}$)
* 1-D transitions profiles bin progress ($\lambda_1/(\lambda_1+\lambda_2)$) only along the transition path ($\lambda_{1}+\lambda_{2}>0.8$)

<img src="Figures/Profile/Bins2.jpg" alf="Bins" width=400/>

* 2-D profiles as a function of $\lambda$ variables at two different sites may be used if there is significant coupling between sites

(See optional "ntersite" parameter for alf module)

1-D profiles consist of 400 bins, 2-D profiles consist of 20x20 bins

### III.A alf.PlotFreeEnergy5() Is Useful For Visualizing Free Energy Profiles

Run help on alf.PlotFreeEnergy5 to learn about usage.

In [ ]:
help(alf.PlotFreeEnergy5)

Plot free energy profiles from early in flattening

In [ ]:
alf.PlotFreeEnergy5('prerun/vacuum/analysis3')

Plot free energy profiles from end of production

In [ ]:
alf.PlotFreeEnergy5('prerun/vacuum/analysis61')

## IV. Biases

Alchemical barriers turn out to have similar shapes, and can be largely flattened with a simple set of biases

### IV.A. Fixed or Linear Bias (b or $\phi$)

With correct parameters, this bias ensures all perturbations at each site are equally populated
$$U_{\text{fixed}}=\sum_{s=1}^M\sum_{i=1}^{N_s} \phi_{si}\lambda_{si}$$

<img src="Figures/Profile/Gphi.jpg" alf="Fixed bias" width=200/>

### IV.B Quadratic Bias (c or $\psi$)

This bias is largely responsible for removing barriers in alchemical space due to electrostatic interactions
$$U_{\text{quad}}=\sum_{s=1}^M\sum_{i=1}^{N_s}\sum_{j=i+1}^{N_s} \psi_{si,sj}\lambda_{si}\lambda_{sj}$$

<img src="Figures/Profile/Gpsi.jpg" alf="Quad bias" width=200/>

### IV.C Skew Bias (x or $\chi$)

Upon the introduction of soft cores the barriers became less symmetric, and this bias fit the residuals beyond the quadratic and end biases well
$$U_{\text{skew}}=\sum_{s=1}^M\sum_{i=1}^{N_s}\sum_{j \neq i}^{N_s} \chi_{si,sj}\lambda_{sj}(1-\exp(\lambda_{si}/0.18))$$

<img src="Figures/Profile/Gchi.jpg" alf="Skew bias" width=200/>

### IV.D End Bias (s or $\omega$)

The end bias pays the entropic and surface tension cost of displacing solvent and nearby molecules to make space for a substituent to appear 
$$U_{\text{end}}=\sum_{s=1}^M\sum_{i=1}^{N_s}\sum_{j \neq i}^{N_s} \omega_{si,sj}\lambda_{sj}\frac{\lambda_{si}}{\lambda_{si}+0.017}$$

<img src="Figures/Profile/Gomega.jpg" alf="End bias" width=200/>

### IV.E Biases Are Stored in Matrices

Each cycle [i] of ALF, previous biases are stored in
* analysis[i]/b_prev.dat
* analysis[i]/c_prev.dat
* analysis[i]/x_prev.dat
* analysis[i]/s_prev.dat

changes to biases are stored in
* analysis[i]/b.dat
* analysis[i]/c.dat
* analysis[i]/x.dat
* analysis[i]/s.dat

and new biases are stored in
* analysis[i]/b_sum.dat
* analysis[i]/c_sum.dat
* analysis[i]/x_sum.dat
* analysis[i]/s_prev.dat

The b, c, x, and s matrices describe the shape of the free energy landscape. Thus, they are opposite in sign from the $\phi$, $\psi$, $\chi$, and $\omega$ parameters required to flatten the landscape.

#### IV.E.1 Final Biases

In [ ]:
print('Fixed bias parameters b_sum.dat')
b=np.loadtxt('prerun/vacuum/analysis61/b_sum.dat')
print(b)
print('Site 1')
print(b[0:5])
print('Site 2')
print(b[5:11])
print('Quad bias parameters c_sum.dat')
c=np.loadtxt('prerun/vacuum/analysis61/c_sum.dat')
print(c)
print('Site 1')
print(c[0:5,0:5])
print('Site 2')
print(c[5:11,5:11])
print('Skew bias parameters x_sum.dat')
x=np.loadtxt('prerun/vacuum/analysis61/x_sum.dat')
print(x)
print('Site 1')
print(x[0:5,0:5])
print('Site 2')
print(x[5:11,5:11])
print('End bias parameters s_sum.dat')
s=np.loadtxt('prerun/vacuum/analysis61/s_sum.dat')
print(s)
print('Site 1')
print(s[0:5,0:5])
print('Site 2')
print(s[5:11,5:11])

Notes:
* b(site 1, sub 1) and b(site 2, sub 1) are zero because $\sum_i^{N_s} \lambda_{si}=1$
* c is upper triangular because quadratic bias is symmetric
* s or $\omega$ biases are near zero because this is the vacuum side of the solvation free energy calculation, and there is no solvent to push out of the way. Typically s biases are a little under 1 kcal/mol per heavy atom

## V. Algorithm Overview

Many cycles of flattening are required because the shape of the free energy profile only becomes clear once biases are near optimal. However, even with poor biases the direction in which to change biases is quite clear from the parts of profiles that are sampled. Thus many cycles of ALF are run. Each cycle consists of molecular dynamics sampling followed by estimation of new bias parameters. Early cycles are very short to avoid wasting computational resources. Simulations get longer as the biases improve.

ALF may be run using the following high level routines:
* alf.initialize()
* alf.runflat()
* alf.runprod()
* alf.postprocess()

Users who desire or require more fine-grained control may examine these high level routines to appropriately call the low level routines upon which they rely.

### V.A. Documentation

The ALF package was documented in detail recently. Read the README.md for an overview of how to use it. Modules and routines are described in docstrings, which can be read by calling `help(alf)` or `help(alf.runflat)` in python. There are several examples of how to use ALF in the examples directory with an INSTRUCTIONS file describing how to run them, and I expect to refine and add examples in the coming weeks.

### V.B. alf.initialize()

Set initial values for biases

In [ ]:
help(alf.initialize)

### V.C. alf.runflat()

Runs several cycles of flattening

In [ ]:
help(alf.runflat)

### V.D. alf.runprod()

Run several chunks of a production run. Saves progress in chunks, usually 1 ns each.

In [ ]:
help(alf.runprod)

### V.E. alf.postprocess()

Reoptimize biases and evaluate $\Delta G$ upon chemical transformation from a production run by alf.runflat. The difference between $\Delta G$ in two different ensembles gives the relative free energy $\Delta \Delta G$.

In [ ]:
help(alf.postprocess)

### V.F. Launching an ALF run

ALF is often run through slurm scripts on a cluster. The file `subsetAll.sh` is designed to submit an entire ALF workflow to the queue and set dependencies between jobs and job array elements appropriately. It uses the following environment variables, which you may set through this tutorial to suit your cluster. `subsetAll.sh` and the slurm scripts it submits may be found in the examples/engines directory of ALF.

In [ ]:
os.environ['SLURMOPTSMD']='--time=240 --ntasks=1 --tasks-per-node=1 --cpus-per-task=1 -p gpu -A rhayes1_lab_gpu --gres=gpu:1 --export=ALL'
os.environ['SLURMOPTSPP']='--time=240 --ntasks=1 --tasks-per-node=1 --cpus-per-task=1 -p gpu -A rhayes1_lab_gpu --gres=gpu:1 --export=ALL'

In [ ]:
os.chdir(alf_tutorial_dir+'/vacuum')
!./subsetAll.sh
os.chdir(alf_tutorial_dir)

One could achieve comparable results without slurm by running

Typically initial flattening runs are 0.1 ns. For this system 50 of these runs are sufficient. 100 cycles is sufficient for most systems. Some strongly charged perturbations like arginine mutations require 200 cycles. Next 10 cycles of 1 ns are typically run to further refine flattening. Then typically 5 independent trials are run for production. In this system and many solvation free energy calculations 10 ns is sufficient. In well behaved ligand and protein mutation systems 20-40 ns is sufficient. In poorly behaved systems 100 ns may be required. Larger numbers of perturbation sites typically increase sampling requirements. The largest system simulated yet with 15 perturbation sites required 12 independent trials of 400 ns for convergence.

## VI. Algorithm Details

### VI.A. Input Format

In ALF, everything system specific is placed inside a directory called prep. The prep directory should contain:
* alf_info.py : a file containing a python dictionary of basic info ALF needs
* 14benz.py : a file setting up the system on the molecular dynamics engine of choice; the name of this file varies
* etc... : all supporting files required by 14benz.py

Detailed instructions for how to prepare these prep directories for several applications of interest will be provided over the next several days.

#### VI.A.1. alf_info.py

In [ ]:
!cat vacuum/prep/alf_info.py

The fields are as follows:
* name : a name for the system; ALF will look for the setup file in name.py
* nsubs : a python list of the number of substituents at each perturbation site; the length of this list is the number of perturbation sites
* nblocks : the total number of perturbations at all sites
* ncentral : for use with replica exchange, set to 0 if not using replica exchange
* nreps : for use with replica exchange, set to 1 if not using replica exchange
* nnodes : for parallelization to multiple GPUs, values greater than 1 do not provide significant benefit for most applications
* enginepath : the path to the molecular dynamics engine executable; not used with pyCHARMM
* temp : the system temperature in Kelvin
* q : an optional field that should be set for charge changing perturbations, which is a list containing the charge of each substituent

#### VI.A.2 14benz.py

In [ ]:
!cat vacuum/prep/14benz.py

This pyCHARMM script proceeds through several of the steps common to most setups on pyCHARMM
* Reading in rtf (residue topology file) and prm (parameter) files
* Generating a psf (protein structure file) by reading sequences (here with read.sequence_pdb)
* Applying alchemical patches (here using lingo to submit commands like `patch p1_1 LIG 1 setup`)
* Defining selections (conventionally with names like site1sub1 to reference alchemical groups)
* Setting up the box (crystal.define_cubic() and image commands)
* Setting up the block module (with lingo)

The block module of CHARMM defines the alchemical portion of the calculation. However, the block module has not been effectively ported to pyCHARMM yet, so it is written as a single large script submitted through lingo. (Lingo allows one to submit commands to pyCHARMM using the original CHARMM scripting language). The "call" commands assign selections of atoms to various alchemical groups. The "ldin" commands set initial conditions, and also read the linear biases. The "ldbi" and "ldbv" commands read in the quadratic, skew, and end biases from their matrices.

#### VI.A.3 etc...

In [ ]:
!ls vacuum/prep

### VI.B. ALF Output

Outside the prep directory is where the output appears. This includes run[i] directories where the molecular dynamics simulations of ALF cycle [i] are performed, analysis[i] directories where new biases are estimated from the sampling recorded in run[i], and variables[i+1].py where these biases are saved to be read into pyCHARMM on the next cycle of dynamics.

In [ ]:
!ls vacuum

### VI.C. Molecular Dynamics

ALF supports several molecular dynamics engines. The engine being used is passed to routines like alf.initialize and alf.runflat as an optional parameter named engine. Currently supported engines include
* charmm : CHARMM script using domdec for gpu acceleration
* bladelib : CHARMM script using BLaDE for gpu acceleration
* blade : BLaDE script using standalone BLaDE
* pycharmm : pyCHARMM python script using BLaDE for gpu acceleration

We hope to eventually support OpenMM. System setup is very similar in other engines, but some python scripts ending in .py become CHARMM scripts ending in .inp. If you are more comfortable with CHARMM scripting than with pyCHARMM, you may wish to consider the charmm or bladelib engines.

You may have noticed there were no energy, minimization, or dynamics commands in prep/14benz.py. That is because the prep/14benz.py script is called by another pyCHARMM script, msld_flat.py for short flattening runs or msld_prod.py for longer production runs. These scripts contain the system independent parts of a $\lambda$ dynamics molecular dynamics simulation. If these scripts do not exist, default versions of them are copied from alf/default_scripts. If the scripts do exist, they are not overwritten, because sometimes small edits are necessary (here I've edited these files to change cutoffs in both ensembles and run constant volume instead of constant pressure in the vacuum ensemble).

In [ ]:
!cat vacuum/msld_flat.py

Note that after loading pyCHARMM, this script begins by running any arguments passed by ALF (arguments.py), reading in the bias and alf_info parameters (variablesflat.py), and setting up the system (prep/14benz.py).

In runflat.py, an optional minimization is performed on the first cycle of ALF. In runflat.py, two segments of dynamics are called, esteps of equilibration and nsteps of sampling. In runprod.py, only one segment of dynamics is called to run a chunk of dynamics, but that chunk starts from the restart file of the previous chunk. An appropriate number of chunks should be discarded for equilibration.

### VI.D. alf.GetLambdas

Walk through a cycle of flattening to see what is going on.

In [ ]:
!ls prerun/vacuum/analysis4 # does not exist before this analysis

In [ ]:
os.chdir(alf_tutorial_dir+'/prerun/vacuum')
alf_info=alf.initialize_alf_info('pycharmm')
G_imp=alf_tutorial_dir+'/prerun/G_imp'
i=4

# Prep the analysis directories
print('analysis%d started' % i)
if not os.path.exists('analysis%d' % i):
  os.mkdir('analysis%d' % i)
shutil.copy('analysis%d/b_sum.dat' % (i-1),'analysis%d/b_prev.dat' % i)
shutil.copy('analysis%d/c_sum.dat' % (i-1),'analysis%d/c_prev.dat' % i)
shutil.copy('analysis%d/x_sum.dat' % (i-1),'analysis%d/x_prev.dat' % i)
shutil.copy('analysis%d/s_sum.dat' % (i-1),'analysis%d/s_prev.dat' % i)
np.savetxt('analysis%d/nsubs' % i,np.array(alf_info['nsubs']).reshape((1,-1)),fmt=' %d')

if not os.path.exists('analysis%d/G_imp' % i):
  if not G_imp:
    G_imp_dir=os.path.dirname(os.path.abspath(__file__))+'/G_imp'
  else:
    G_imp_dir=G_imp
  os.symlink(G_imp_dir,'analysis%d/G_imp' % i)
os.chdir('analysis%d' % i)

# Run the analysis
alf.GetLambdas(alf_info,i)

os.chdir(alf_tutorial_dir)

* Copy previous biases into current analysis directory (in this case analysis4).
* Link the G_imp directory to a precomputed directory. The implicit constraints are a nonlinear mapping from $\theta$ space to $\lambda$ space, so sampling will be non-uniform in $\lambda$ space due to implicit constraints. $\lambda$ values near physical end states are sampled more. Subtract the free energy of implicit constraints before flattening so we don't get rid of this desirable feature by flattening
* alf.GetLambdas copies alchemical trajectory from binary format in run4/res/14benz_flat.lmd into human readable format in analysis4/data/Lambda.0.0.dat

<img src="Figures/ImpCons/G1_6.jpg" alf="Gimp" width=250/>

In [ ]:
!ls prerun/vacuum/analysis4/data
!head prerun/vacuum/analysis4/data/Lambda.0.0.dat

### VI.E. alf.GetEnergy

In early testing of ALF, sometimes ALF would change a biasing parameter, sampling would shift, and ALF would immediately change it back, because it effectively had no memory of why it made changes in previous cycles. To deal with this, sampling from the previous 5 cycles is agregated together. This gives a more comprehensive view of the landscape. Including significantly more than the previous 5 cycles was found to cause artifacts because older simulations far from equilibrium were treated on the same footing as more recent simulations with better biases, leading to distorted estimates of the landscape. The previous 5 cycles were all run with different biasing potentials, so they are combined with WHAM/MBAR. The WHAM/MBAR equations require the energies of each alchemical trajectory in each of the 5 biases that were used. alf.GetEnergy computes these energies.

In [ ]:
os.chdir(alf_tutorial_dir+'/prerun/vacuum/analysis4')
im5=1

alf.GetEnergy(alf_info,im5,i)

os.chdir(alf_tutorial_dir)

The previous 5 alchemical trajectories are copied into analysis4/Lambda, and the relative energies are saved in analysis4/Energy.

In [ ]:
!ls prerun/vacuum/analysis4/Lambda
!ls prerun/vacuum/analysis4/Energy

### VI.F. alf.RunWham

alf.RunWham is a python wrapper for the GPU code compiled in the alf/wham directory during installation. This code does most of the heavy lifting of the bias optimization.

First, the output of alf.GetEnergy is used to reweight all the alchemical trajectory frames from the the previous 5 (or in this case 4) simulations using WHAM/MBAR.

Next, free energy profiles like those plotted above are computed by binning.

A penalty function
$$E=\sum_p^{\text{Profiles}}\sum_b^{\text{Bins}}\frac{k_{pb}}{2}(G_{pb}-G_{pb}^{\text{imp}}-\bar{G}_p)^2$$
penalizes deviations of each point $G_{pb}$ at bin $b$ on profile $p$ from the average value of that profile $\bar{G}_p$ (after subtracting off the implicit constraint free energy $G_{pb}^{\text{imp}}$ for that point). The $k_{pb}$ term is used to more tightly restrain some profiles (the 1-D profiles), and some bins (the endpoint bins) to the average value.

Changing the bias parameters changes the weights output by WHAM/MBAR, which in turn changes $G_{pb}$. The effect of the bias parameters on each $G_{pb}$ is nonlinear, which gives rise to a nonlinear optimization problem. To simplify the problem, the nonlinear dependence of $G_{pb}$ on the bias parameters is approximated as linear. If the bias parameters $\phi$, $\psi$, $\chi$, and $\omega$ are collectively referred to as $\alpha_i$, $G_{pb}$ may be approximated as
$$G_{pb}(\{\alpha_i\}) \approx G_{pb} + \sum_i^{\text{Biases}}\Delta \alpha_i \frac{\partial G_{pb}}{\partial \alpha_i}$$
A regularization term $\sum_i 0.5 k_i \Delta {\alpha_i}^2$ is added by the next routine alf.GetFreeEnergy5 to prevent excessively large changes in the biases.

This reduces the penalty function $E$ to a quadratic function of the changes in the biases $\Delta \alpha_i$, and the optimum of this quadratic function is given by a system of linear equations
$$C\Delta\alpha=V$$
The matrix $C$ and vector $V$ are written to analysis4/multisite/C.dat and analysis4/multisite/V.dat.

In [ ]:
os.chdir(alf_tutorial_dir+'/prerun/vacuum/analysis4')
N=i-im5+1
ntersite=[0,0]

fpout=open('output','w')
fperr=open('error','w')
subprocess.call([shutil.which('python'),'-c','import alf; alf.RunWham(%d,%f,%d,%d)' % (N*alf_info['nreps'],alf_info['temp'],ntersite[0],ntersite[1])],stdout=fpout,stderr=fperr)

os.chdir(alf_tutorial_dir)

In [ ]:
!ls prerun/vacuum/analysis4/multisite/C.dat
!ls prerun/vacuum/analysis4/multisite/V.dat

### VI.G. alf.GetFreeEnergy5

alf.GetFreeEnergy5 takes the $C$ matrix and $V$ vector given by alf.RunWham, adds regularization terms, inverts $C$, and solves for $\Delta \alpha$. If any $\alpha_i$ parameter changes by more than a maximum value, all changes are scaled down such that no change exceeds its cap. Changes are saved to analysis4/b.dat, analysis4/c.dat, analysis4/x.dat, and analysis4/s.dat

In [ ]:
os.chdir(alf_tutorial_dir+'/prerun/vacuum/analysis4')

alf.GetFreeEnergy5(alf_info,ntersite[0],ntersite[1])

os.chdir(alf_tutorial_dir)

In [ ]:
!cat prerun/vacuum/analysis4/b.dat

Note that substituents 2 and 3 at both sites 1 and 2 which were oversampled have negative b changes to drive sampling to other substituents.

### VI.H. alf.SetVars

With the hard work done, alf.SetVars saves the bias variables into a format readable by the MD engine through msld_flat.py

In [ ]:
os.chdir(alf_tutorial_dir+'/prerun/vacuum/analysis4')

alf.SetVars(alf_info,i+1)

os.chdir(alf_tutorial_dir)

In [ ]:
!ls prerun/vacuum/variables5.py
!head prerun/vacuum/variables5.py

### VI.I. Production and Postprocessing

In flattening by alf.runflat, dynamics and bias optimization alternate many times within a single slurm script. In production, alf.runprod performs dynamics and alf.postprocess performs bias optimization and free energy estimation.

Typically 5 independent trials are run by alf.runprod during production using the same biases in order to estimate the statistical precision of the results. Each of these simulations is run with a different tag, a, b, c, d, or e. In addition, each production is divided into several cycles, typically of 1 ns each, and each call of alf.runprod is responsible for some number of these cycles (in this case 5). This makes it easy to submit production simulations that would run longer than a cluster time limit with a slurm job array where each element is significantly shorter than the time limit.

alf.postprocess follows largely the same flattening algorithm as alf.runflat, but in addition uses the routine alf.GetVariance to estimate free energies. Free energies are estimated by
$$G_i - G_0 = -k_BT \ln(P_i/P_0) -(\phi_i-\phi_0)$$
where $P_i$ is the fraction of the time ligand $i$ was sampled, $P_0$ is the fraction of the time the reference ligand was sampled, and the excess free energy contributed by the biases to each state is subtracted off. Statistical uncertainties for these free energies are estimated by bootstrapping from the 5 independent trials.

### VI.J. Robustness

The implementation of ALF is designed to be robust against interruptions such as simulation crashes and node failure. During alf.runflat, each cycle is repeatedly attempted until it succeeds (judged by the presence of analysis[i]/b.dat). If the slurm job goes down, alf.runflat may be relaunched and ALF will resume at the first incomplete cycle. During alf.runprod, each 1 ns chunk is attempted until it succeeds (judged by the presence of an alchemical trajectory containing the correct number of frames), and if previous instances of alf.runprod did not complete their chunks, the current instance will go back and redo their chunks.

## VII. Results

The $\Delta G$ for an alchemical process in one physical ensemble given by alf.postprocess using alf.GetVariance has little meaning unless it is compared to the $\Delta G$ of that same chemical process in another physical ensemble. The difference $\Delta \Delta G$ is the relative free energy of interest.

<img src="Figures/CycleBoth.jpg" alf="cycle" width=400/>

In [ ]:
!echo vacuum
!cat prerun/vacuum/analysis61/Result.txt
!echo solvent
!cat prerun/solvent/analysis61/Result.txt

import math

fp1=open("prerun/vacuum/analysis61/Result.txt","r")
fp2=open("prerun/solvent/analysis61/Result.txt","r")
fp3=open("prerun/Result.txt","w")

lines1=fp1.readlines()
lines2=fp2.readlines()

nsites=len(lines1[0].split())-3
for i in range(0,len(lines1)):
  line1=lines1[i].split()
  line2=lines2[i].split()

  i1=[]
  for j in range(0,nsites):
    i1.append(int(line1[j]))
  V=float(line2[nsites])-float(line1[nsites])
  E=math.sqrt(float(line2[nsites+2])**2 + float(line1[nsites+2])**2)

  for j in range(0,nsites):
    fp3.write("%2d " % (i1[j],))
  fp3.write("%8.3f +/- %5.3f\n" % (V,E))

fp1.close()
fp2.close()
fp3.close()

!echo ddG
!cat prerun/Result.txt

## VIII. Further Practice

* Run the solvent side of the calculation 1,4-substituted-benzene solvation free energy calculation in the solvent directory
* Read the INSTRUCTIONS file in the ALF respository examples directory, and use the bladelib engine to run the unfolded side of T4 lysozyme (from the directory examples/systems/prep_T4L149U_charmm)